In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import nltk
import string

In [2]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect(r'F:\Data Science projects\By_me\ML\Amazon\Dataset/database.sqlite')

OperationalError: unable to open database file

In [ ]:
type(con)

#### reading data from Sqlite database

In [ ]:
pd.read_sql_query("SELECT * FROM Reviews", con)

In [ ]:
df=pd.read_sql_query("SELECT * FROM Reviews", con)

#### reading some n number of rows, use LIMIT over ther

In [ ]:
pd.read_sql_query("SELECT * FROM Reviews LIMIT 3", con)

#### or we can also Load the dataset using pandas

In [ ]:
df = pd.read_csv('F:\Data Science projects\By_me\ML\Amazon\Dataset/Reviews.csv')

print(df.shape)
df.head()

In [ ]:
df.shape

### What is sentiment analysis?
    Sentiment analysis is the computational task of automatically determining what feelings a writer is expressing in text
    Some examples of applications for sentiment analysis include:

    1.Analyzing the social media discussion around a certain topic
    2.Evaluating survey responses
    3.Determining whether product reviews are positive or negative

    Sentiment analysis is not perfect.It also cannot tell you why a writer is feeling a certain way. However, it can be useful to quickly summarize some qualities of text, especially if you have so much text that a human reader cannot analyze it.For this project,the goal is to to classify Food reviews based on customers' text.

In [ ]:
from textblob import TextBlob

In [ ]:
TextBlob(df['Summary'][0]).sentiment.polarity

In [ ]:
## takes 3 mins 
polarity=[] # list which will contain the polarity of the comments

for i in df['Summary']:
    try:
        polarity.append(TextBlob(i).sentiment.polarity)   
    except:
        polarity.append(0)

In [ ]:
len(polarity)

In [ ]:
data=df.copy()

In [ ]:
data['polarity']=polarity

In [ ]:
data.head()

In [ ]:
data['polarity'].nunique()

### Lets perform EDA for the Positve sentences¶

In [ ]:
data_positive = data[data['polarity']>0]
data_positive.shape

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
stopwords=set(STOPWORDS)

In [ ]:
positive=data_positive[0:200000]

In [ ]:

total_text= (' '.join(data_positive['Summary']))


In [ ]:
len(total_text)

In [ ]:
total_text[0:10000]

In [ ]:
import re
total_text=re.sub('[^a-zA-Z]',' ',total_text)

In [ ]:
total_text

In [ ]:
## remove extra spaces
total_text=re.sub(' +',' ',total_text)

In [ ]:
total_text[0:10000]

In [ ]:
len(total_text)

In [ ]:
wordcloud = WordCloud(width = 1000, height = 500,stopwords=stopwords).generate(total_text)
plt.figure(figsize=(15,5))
plt.imshow(wordcloud)
plt.axis('off')

## Lets perform EDA for the Neagtive sentences

In [ ]:
data_negative = data[data['polarity']<0]
data_negative.shape

In [ ]:
data_negative.head()

In [ ]:
total_negative= (' '.join(data_negative['Summary']))


In [ ]:
total_negative

In [ ]:
import re
total_negative=re.sub('[^a-zA-Z]',' ',total_negative)

In [ ]:
len(total_negative)

In [ ]:
total_negative

In [ ]:
total_negative=re.sub(' +',' ',total_negative)

In [ ]:
len(total_negative)

In [ ]:

wordcloud = WordCloud(width = 1000, height = 500,stopwords=stopwords).generate(total_negative)
plt.figure(figsize=(15,5))
plt.imshow(wordcloud)
plt.axis('off')

## Analyse to what User Amazon Can recommend more product

#### Amazon can recommend more products to only those who are going to buy more or to one who has a better conversion rate,so lets ready data according to this problem statement

In [ ]:
df['UserId'].shape

In [ ]:
df['UserId'].nunique()

In [ ]:
df.head()

In [ ]:
raw=df.groupby(['UserId']).agg({'Summary':'count', 'Text':'count','Score':'mean','ProductId':'count'}).sort_values(by='Text',ascending=False)
raw

In [ ]:
raw.columns=['Number_of_summaries','num_text','Avg_score','Number_of_products_purchased']
raw

In [ ]:
user_10=raw.index[0:10]
number_10=raw['Number_of_products_purchased'][0:10]

plt.bar(user_10, number_10, label='java developer')
plt.xlabel('User_Id')
plt.ylabel('Number of Products Purchased')
plt.xticks(rotation='vertical')

#### These are the Top 10 Users so we can recommend more & more Prodcuts to these Usser Id as there will be a high probability that these person are going to be buy more

#### as data is so huge,so if your system takes a lot for the execution , u can considered some sample of data from entire data,
    as may be some of you have not that much good specifications in terms of processor ,RAM & HArd Disk..
    so according to system specifications,u can considered some sample of data,if u have not issue with your specifications,
    u can go ahead with this bulky data

In [ ]:
df.head()

In [ ]:
## picking a random sample
final=df.sample(n=2000)

In [ ]:
final=df[0:2000]

#### check missing values in dataset

In [ ]:
final.isna().sum()

#### Removing the Duplicates if any

In [ ]:
final.duplicated().sum()

### Analyse Length of Comments whether Customers are going to give Lengthy comments or short one

In [ ]:
final.head()

In [ ]:
len(final['Text'][0].split(' '))

In [ ]:
final['Text'][0]

In [ ]:

def calc_len(text):
    return (len(text.split(' ')))

In [ ]:
final['Text_length']=final['Text'].apply(calc_len)

In [ ]:
import plotly.express as px
px.box(final, y="Text_length")

#### Conclusion-->>
    Seems to have Almost 50 percent users are going to give their Feedback limited to 50 words whereas there are only few users who are going give Lengthy Feedbacks

#### Analyze Score

In [ ]:
sns.countplot(final['Score'], palette="plasma")

### Text Pre-Processsing

In [ ]:
final['Text'] =final['Text'].str.lower()
final.head(10)

In [ ]:
final['Text'][164]

In [ ]:
import re
re.sub('[^a-zA-Z]',' ',final['Text'][164])

#### drawback of this re.sub in this use-case is, it will remove some numerical data too & may be that numerical values matters alot
#### thats way, I am going to create my own logic over here,that will remove all the special character

#### logic to remove punctuations or all the special characters

In [ ]:
# define punctuation
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

data= final['Text'][164]

# remove punctuation from the string
no_punct = ""
for char in data:
    if char not in punctuations:
        no_punct = no_punct + char

# display the unpunctuated string
no_punct

#### Create function to remove punctuations in your review

In [ ]:
def remove_punc(review):
    import string
    punctuations =string.punctuation
    # remove punctuation from the string
    no_punct = ""
    for char in review:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [ ]:
final['Text'] =final['Text'].apply(remove_punc)

In [ ]:
final.head()

In [ ]:
final['Text'][164]

#### Removal of Stopwords

In [ ]:
import re
from nltk.corpus import stopwords

In [ ]:
review='seriously this product was as tasteless as they come there are much better tasting products out there but at 100 calories its better than a special k bar or cookie snack pack you just have to season it or combine it with something else to share the flavor'

In [ ]:
re=[word for word in review.split(' ') if word not in set(stopwords.words('english'))]
str=''
for wd in re:
    str=str+wd
    str=str+' '
str

#### using join to convert list into string

In [ ]:
re=[word for word in review.split(' ') if word not in set(stopwords.words('english'))]
' '.join(re)

#### perform this task using function as I have to apply this logic on my entire column

In [ ]:
def remove_stopwords(review):
    return ' '.join([word for word in review.split(' ') if word not in set(stopwords.words('english'))])

In [ ]:
remove_stopwords(review)

In [ ]:
final.shape

In [ ]:
final.columns

In [ ]:
final['Text'] = final['Text'].apply(remove_stopwords)

In [ ]:
final.head()

### Pre-process your Data in a Depth

#### check if urls is present in Text column or not

In [ ]:
final['Text'].str.contains('http?').sum()

In [ ]:
final['Text'].str.contains('http').sum()

In [ ]:
pd.set_option('display.max_rows',2000)
final['Text'].str.contains('http',regex=True)

In [ ]:
final['Text'][21]

#### we will observe we have some kind of URLs over here in my data that is definitely a kind of Dirtines in data, so we have to clean this data & make ready data for the analysis purpose

####  Removal of urls

In [8]:
final['Text'][21]

NameError: name 'final' is not defined

In [ ]:
review=final['Text'][21]
review

In [ ]:
import re

In [ ]:
url_pattern = re.compile(r'href|http.\w+')
url_pattern.sub(r'', review)

In [ ]:
import re
def remove_urls(review):
    url_pattern = re.compile(r'href|http.\w+')
    return url_pattern.sub(r'', review)

In [ ]:
final['Text'] = final['Text'].apply(remove_urls)


In [ ]:
final.head()


In [ ]:
final['Text'].str.contains('http').sum()

In [ ]:
final['Text'][34]

##### as we will see we have lots of br in my data, let me remove wherever i have br

In [ ]:
final['Text'][34].replace('br','')


In [ ]:
for i in range(len(final['Text'])):
    final['Text'][i]=final['Text'][i].replace('br','')

In [ ]:
data2=final.copy()

In [ ]:
data2['Text'][34]

In [ ]:
data2.shape

In [ ]:
data2.dtypes

In [ ]:
from wordcloud import WordCloud, STOPWORDS 

In [ ]:
stopwords = set(STOPWORDS) 

In [ ]:
data2.head()

In [ ]:
comment_words = '' 
for val in data2['Text']:
    # typecaste each val to string
    
    # split the value 
    tokens = val.split() 
    
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
    comment_words=comment_words+ " ".join(tokens)+" "
    

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 

In [ ]:
# plot the WordCloud image                        
plt.figure(figsize = (8, 8)) 
plt.imshow(wordcloud) 
plt.axis("off") 